In [ ]:
sig_table = np.zeros(shape=(len(X_train.columns)))
signs_table = np.zeros(shape=(len(X_train.columns)))
kfold = KFold(n_splits=numCV, shuffle=True)

#used for pcorr kfolds as well as best subsets
train_ = []
test_ = []

kfold.get_n_splits(X_train.index)

for train_indices, test_indices in kfold.split(X_train.index):
    train_.append(train_indices)
    test_.append(test_indices)

In [ ]:
sig_table = np.zeros(len(X_train.columns))
signs_table = np.zeros(len(X_train.columns))
significance = np.zeros(len(X_train.columns))
sign = np.zeros(len(X_train.columns))
purity = np.zeros(len(X_train.columns))

for it in range(0,len(train_)):
    max_pvalue = 1
    train_index = train_[it]
    test_index = test_[it]
    #display(X_train.iloc[test_index].describe())
    
    subset = pd.concat([X_train[filterthrough],y_train],axis=1)
    #subset = X_train.iloc[train_index].loc[:, ~X_train.columns.isin([exclude])]
    
    #skip y and states
    set_ = filterthrough
    #set_ = subset.loc[:, ~subset.columns.isin([target])].columns.tolist()
    
    n=len(subset)
    
    while(max_pvalue>=.05):

        dist = scipy.stats.beta(n/2 - 1, n/2 - 1, loc=-1, scale=2)
        p_values = pd.DataFrame(2*dist.cdf(-abs(subset.pcorr()[target]))).T
        p_values.columns = list(subset.columns)
        
        max_pname = p_values.idxmax(axis=1)[0]
        max_pvalue = p_values[max_pname].values[0]
        
        if (max_pvalue > .05):

            set_.remove(max_pname)
            temp = [target]
            temp.extend(set_)
            subset = subset[temp]
    
    winners = p_values.loc[:, ~p_values.columns.isin([target])].columns.tolist()
    sig_table = (sig_table + np.where(X_train.columns.isin(winners),1,0)).copy()
    signs_table[X_train.columns.get_indexer(winners)]+=np.where(subset.pcorr()[target][winners]<0,-1,1)
    
significance = pd.DataFrame(sig_table).T
significance.columns = list(X_train.columns)
display(significance)

sign = pd.DataFrame(signs_table).T
sign.columns = list(X_train.columns)
display(sign)

purity = abs((sign/numCV)*(sign/significance)).T.replace([np.inf, -np.inf, np.NaN], 0)
display(purity.T)

threshold = 0

chosen = list(purity.T.columns.values[np.array(purity.T>threshold).reshape(len(X_train.columns,))])

display(chosen)    